Użyj każdą z tych funkcji 
* `unix_timestamp()` 
* `date_format()`
* `to_unix_timestamp()`
* `from_unixtime()`
* `to_date()` 
* `to_timestamp()` 
* `from_utc_timestamp()` 
* `to_utc_timestamp()`

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, current_timestamp

spark = SparkSession.builder.appName("TestApp").getOrCreate()

kolumny = ["timestamp", "unix", "Date"]
dane = [
    ("2015-03-22T14:13:34", 1646641525847, "May, 2021"),
    ("2015-03-22T15:03:18", 1646641557555, "Mar, 2021"),
    ("2015-03-22T14:38:39", 1646641578622, "Jan, 2021")
]

dataFrame = spark.createDataFrame(dane, kolumny) \
    .withColumn("current_date", current_date()) \
    .withColumn("current_timestamp", current_timestamp())

display(dataFrame)
dataFrame.show()


25/03/26 15:41:36 WARN Utils: Your hostname, MacBook-Air-Witold.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
25/03/26 15:41:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 15:41:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[timestamp: string, unix: bigint, Date: string, current_date: date, current_timestamp: timestamp]

+-------------------+-------------+---------+------------+--------------------+
|          timestamp|         unix|     Date|current_date|   current_timestamp|
+-------------------+-------------+---------+------------+--------------------+
|2015-03-22T14:13:34|1646641525847|May, 2021|  2025-03-26|2025-03-26 15:41:...|
|2015-03-22T15:03:18|1646641557555|Mar, 2021|  2025-03-26|2025-03-26 15:41:...|
|2015-03-22T14:38:39|1646641578622|Jan, 2021|  2025-03-26|2025-03-26 15:41:...|
+-------------------+-------------+---------+------------+--------------------+



In [ ]:

dataFrame.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: timestamp (nullable = false)



## unix_timestamp(..) & cast(..)

Konwersja **string** to a **timestamp**.

Lokalizacja funkcji 
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

## 1. Zmiana formatu wartości timestamp yyyy-MM-dd'T'HH:mm:ss 
`unix_timestamp(..)`

Dokumentacja API `unix_timestamp(..)`:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

In [ ]:
 # `unix_timestamp()` 
import pyspark.sql.functions as F
from pyspark.sql.functions import *

selected = dataFrame.select("timestamp")
dataFrame = dataFrame.withColumn("unix_timestamp",F.unix_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss"))
dataFrame.show()



+-------------------+-------------+---------+------------+--------------------+--------------+
|          timestamp|         unix|     Date|current_date|   current_timestamp|unix_timestamp|
+-------------------+-------------+---------+------------+--------------------+--------------+
|2015-03-22T14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:...|    1427033614|
|2015-03-22T15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:...|    1427036598|
|2015-03-22T14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:...|    1427035119|
+-------------------+-------------+---------+------------+--------------------+--------------+



2. Zmień format zgodnie z klasą `SimpleDateFormat`**yyyy-MM-dd HH:mm:ss**
  * a. Wyświetl schemat i dane żeby sprawdzicz czy wartości się zmieniły

In [ ]:
# `date_format()`
zmianaFormatu = dataFrame.withColumn("timestamp", date_format(col("timestamp"), "yyyy-MM-dd HH:mm:ss")).withColumn("current_timestamp", date_format(col("current_timestamp"), "yyyy-MM-dd HH:mm:ss"))
# with.Column() 
zmianaFormatu.printSchema()
zmianaFormatu.show()

root
 |-- timestamp: string (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: string (nullable = false)
 |-- unix_timestamp: long (nullable = true)

+-------------------+-------------+---------+------------+-------------------+--------------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|
+-------------------+-------------+---------+------------+-------------------+--------------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:07|    1427033614|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:07|    1427036598|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:07|    1427035119|
+-------------------+-------------+---------+------------+-------------------+--------------+



In [ ]:
#unix_timestamp  
tempE = zmianaFormatu.withColumn("nowy_unix_timestamp", unix_timestamp("current_date", "yyyy-MM-dd"))
tempE.show()
display(tempE)

+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:08|    1427033614|         1742342400|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:08|    1427036598|         1742342400|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:08|    1427035119|         1742342400|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+



timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:09,1427033614,1742342400
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:09,1427036598,1742342400
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:09,1427035119,1742342400


## Stwórz nowe kolumny do DataFrame z wartościami year(..), month(..), dayofyear(..)

In [ ]:
#date_format
dataFrame = tempE

yearDate = dataFrame.withColumn("year", date_format(col("timestamp"), "yyyy")) \
    .withColumn("month", date_format(col("timestamp"), "MM")) \
        .withColumn("dayofyear", date_format(col("timestamp"), "dd"))

yearDate.show()
        
# 

+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|year|month|dayofyear|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:10|    1427033614|         1742342400|2015|   03|       22|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:10|    1427036598|         1742342400|2015|   03|       22|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:10|    1427035119|         1742342400|2015|   03|       22|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+



In [ ]:
#to_date()  
toDate = dataFrame.withColumn("year", year(to_date("current_timestamp"))) \
    .withColumn("month", month(to_date("current_timestamp"))) \
    .withColumn("dayofyear", dayofyear(to_date("current_timestamp")))
display(toDate)
toDate.show()

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp,year,month,dayofyear
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:11,1427033614,1742342400,2025,3,78
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:11,1427036598,1742342400,2025,3,78
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:11,1427035119,1742342400,2025,3,78


+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|year|month|dayofyear|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:12|    1427033614|         1742342400|2025|    3|       78|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:12|    1427036598|         1742342400|2025|    3|       78|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:12|    1427035119|         1742342400|2025|    3|       78|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+----+-----+---------+



In [ ]:
# G	era designator	Text	AD
# y	year	Number	2009
# M	month in year	Text & Number	July & 07
# d	day in month	Number	10
# h	hour in am/pm (1-12)	Number	12
# H	hour in day (0-23)	Number	0
# m	minute in hour	Number	30
# s	second in minute	Number	55
# S	millisecond	Number	978
# E	day in week	Text	Tuesday
# D	day in year	Number	189
# F	day of week in month	Number	2 (2nd Wed in July)
# w	week in year	Number	27
# W	week in month	Number	2
# a	am/pm marker	Text	PM
# k	hour in day (1-24)	Number	24
# K	hour in am/pm (0-11)	Number	0
# z	time zone	Text	Pacific Standard Time
# '	escape for text	Delimiter	(none)
# '	single quote	Literal	'

In [ ]:
#from_unixtime()
fromUnix = dataFrame.withColumn("year_UNIX", from_unixtime(col("unix")/1000, "yyyy")) \
    .withColumn("month_UNIX", from_unixtime(col("unix")/1000, "MM")) \
        .withColumn("day_UNIX", from_unixtime(col("unix")/1000, "DDD"))
fromUnix.show()
display(fromUnix)

+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+---------+----------+--------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|year_UNIX|month_UNIX|day_UNIX|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+---------+----------+--------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:13|    1427033614|         1742342400|     2022|        03|     066|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:13|    1427036598|         1742342400|     2022|        03|     066|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:13|    1427035119|         1742342400|     2022|        03|     066|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+---------+----------+--

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp,year_UNIX,month_UNIX,day_UNIX
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:13,1427033614,1742342400,2022,03,066
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:13,1427036598,1742342400,2022,03,066
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:13,1427035119,1742342400,2022,03,066


In [ ]:
#to_timestamp()
toTimestamp = dataFrame.withColumn("timestamp_YEAR", year(to_timestamp(col("timestamp")))) \
    .withColumn("timestamp_MONTH", month(to_timestamp(col("timestamp")))) \
        .withColumn("timestamp_DAY", dayofyear(to_timestamp(col("timestamp"))))
display(toTimestamp)
toTimestamp.show()

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp,timestamp_YEAR,timestamp_MONTH,timestamp_DAY
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:14,1427033614,1742342400,2015,3,81
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:14,1427036598,1742342400,2015,3,81
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:14,1427035119,1742342400,2015,3,81


+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+--------------+---------------+-------------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|timestamp_YEAR|timestamp_MONTH|timestamp_DAY|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+--------------+---------------+-------------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:15|    1427033614|         1742342400|          2015|              3|           81|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:15|    1427036598|         1742342400|          2015|              3|           81|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:15|    1427035119|         1742342400|          2015|              3|           81|
+-------------------+-------------+---------+-

In [ ]:
#to_utc_timestamp()
toUtcTimestamp = dataFrame.withColumn("timestamp_YEAR", year(to_utc_timestamp(col("timestamp"), tz = "UTC"))) \
    .withColumn("timestamp_MONTH", month(to_utc_timestamp(col("timestamp"), tz = "UTC"))) \
        .withColumn("timestamp_DAY", dayofyear(to_utc_timestamp(col("timestamp"), tz = "UTC")))
display(toUtcTimestamp)
toUtcTimestamp.show()


timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp,timestamp_YEAR,timestamp_MONTH,timestamp_DAY
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:16,1427033614,1742342400,2015,3,81
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:16,1427036598,1742342400,2015,3,81
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:16,1427035119,1742342400,2015,3,81


+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+--------------+---------------+-------------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|timestamp_YEAR|timestamp_MONTH|timestamp_DAY|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+--------------+---------------+-------------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:17|    1427033614|         1742342400|          2015|              3|           81|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:17|    1427036598|         1742342400|          2015|              3|           81|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:17|    1427035119|         1742342400|          2015|              3|           81|
+-------------------+-------------+---------+-

In [ ]:
#from_utc_timestamp()
fromUtcTimestamp = dataFrame.withColumn("curr_timestamp_YEAR", year(from_utc_timestamp(col("current_timestamp"), tz = "UTC"))) \
    .withColumn("curr_timestamp_MONTH", month(from_utc_timestamp(col("current_timestamp"), tz = "UTC"))) \
        .withColumn("curr_timestamp_DAY", dayofyear(from_utc_timestamp(col("current_timestamp"), tz = "UTC")))
display(fromUtcTimestamp)
fromUtcTimestamp.show()

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,nowy_unix_timestamp,curr_timestamp_YEAR,curr_timestamp_MONTH,curr_timestamp_DAY
2015-03-22 14:13:34,1646641525847,"May, 2021",2025-03-19,2025-03-19 01:22:18,1427033614,1742342400,2025,3,78
2015-03-22 15:03:18,1646641557555,"Mar, 2021",2025-03-19,2025-03-19 01:22:18,1427036598,1742342400,2025,3,78
2015-03-22 14:38:39,1646641578622,"Jan, 2021",2025-03-19,2025-03-19 01:22:18,1427035119,1742342400,2025,3,78


+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+-------------------+--------------------+------------------+
|          timestamp|         unix|     Date|current_date|  current_timestamp|unix_timestamp|nowy_unix_timestamp|curr_timestamp_YEAR|curr_timestamp_MONTH|curr_timestamp_DAY|
+-------------------+-------------+---------+------------+-------------------+--------------+-------------------+-------------------+--------------------+------------------+
|2015-03-22 14:13:34|1646641525847|May, 2021|  2025-03-19|2025-03-19 01:22:19|    1427033614|         1742342400|               2025|                   3|                78|
|2015-03-22 15:03:18|1646641557555|Mar, 2021|  2025-03-19|2025-03-19 01:22:19|    1427036598|         1742342400|               2025|                   3|                78|
|2015-03-22 14:38:39|1646641578622|Jan, 2021|  2025-03-19|2025-03-19 01:22:19|    1427035119|         1742342400|               20

In [ ]:
%fs ls dbfs:/databricks-datasets/


path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,0
dbfs:/databricks-datasets/README.md,README.md,976,1532468253000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1455043490000
dbfs:/databricks-datasets/adult/,adult/,0,0
dbfs:/databricks-datasets/airlines/,airlines/,0,0
dbfs:/databricks-datasets/amazon/,amazon/,0,0
dbfs:/databricks-datasets/asa/,asa/,0,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,0
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,0
